# Introduction

## Using this notebook

This notebook helps you to analyze datasets and to find interesting and meaningful patterns in the data. If you are only interested in looking at an automated report outlining the most important features of your dataset, you can upload your datafile via the *dataset* variable and run the notebook. Afterwards, you can export the report as HTML and read it in a webbrowser.

If you are interested in a more interactive analysis of your data, you can also adapt the parameters of the notebook to suit your needs. Each section conatins several values which can be adapted to your needs. These values are described in the code comments.

Finally, if you want to go beyond an automated report and answer your own questions, you can look at the final section of the notebook and use the code examples there to generate your own figures and analysis from the data model.

### Reading this report in a webbrowser

This report uses several statistical methods and specific phrases and concepts from the domains of statistics and machine learning. Whenever such methods are used, a small "Explanation" sign at the side of the report marks a short explanation of the methods and phrases. Clicking it will reveal the explanation.

You can toggle the global visibility of these explanations with a button at the top left corner of the report. The code can also be toggled with a button.

All graphs are interactive and will display additional content on hover. You can get the exact values of the functions by selecting the assoziated areas in the graph. You can also move the plots around and zoom into interesting parts.

### Aknowledgments

This notebook is build on the MSPN implementation by Molina et.al. during the course of a bachelor thesis under the supervision of Alejandro Molina and Kristian Kersting at TU Darmstadt. The goal of this framework is to sum product networks for hybrid domains and to highlight important aspects and interesting features of a given dataset.

In [1]:
import pickle

import ba_functions as f
import ba_plot as p
import ba_descriptions as descr
import pandas as pd
import numpy as np

#from tfspn.SPN import SPN
from pprint import PrettyPrinter
from IPython.display import Image
from IPython.display import display, Markdown
from importlib import reload

import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ba_functions import printmd, query, predict_proba, predict

from spn.algorithms.LearningWrappers import learn_piecewise_from_data

init_notebook_mode(connected=True)
pp = PrettyPrinter()

In [2]:
# path to the dataset you want to use for training
dataset = 'example_data/titanic.csv'

# the minimum number of datapoints that are included in a child of a 
# sum node
precision = 50

# the parameter which governs how strict the independece test will be
# 1 results in all features being evaluated as independent, 0 will 
# result in no features being acccepted as truly independent
independence = 0.1


spn, dictionary, context = learn_piecewise_from_data(
    data_file=dataset, 
    header=0, 
    min_instances=25, 
    independence_threshold=0.1, 
    histogram=True)
df = pd.read_csv(dataset)
categoricals = context.get_categoricals()

/home/iliricon/.virtualenvs/SimpleSPN/lib/python3.7/site-packages/sklearn/cross_decomposition/pls_.py:313: UserWarning:

X scores are null at iteration 0

/home/iliricon/.virtualenvs/SimpleSPN/lib/python3.7/site-packages/numpy/lib/function_base.py:2400: RuntimeWarning:

invalid value encountered in true_divide



In [3]:
# path to the model pickle file
model_path = "models/test.pickle"

# UNCOMMENT THE FOLLOWING LINES TO LOAD A MODEL
#spn = SPN.from_pickle(model_path)
#dictionary = pickle.load(model_path + "_d")

#df = pd.read_csv(data_path)

In [4]:
reload(descr)
descr.introduction(spn)

# Exploring the SumNode_0 dataset

<figure align="right" style="padding: 1em; float:right; width: 300px">
	<img alt="the logo of TU Darmstadt"
		src="images/tu_logo.gif">
	<figcaption><i>Report framework created @ TU Darmstadt</i></figcaption>
        </figure>
This report describes the dataset SumNode_0 and contains general statistical
information and an analysis on the influence different features and subgroups
of the data have on each other. The first part of the report contains general
statistical information about the dataset and an analysis of the variables
and probability distributions.<br/>
The second part focusses on a subgroup analysis of the data. Different
clusters identified by the network are analyzed and compared to give an
insight into the structure of the data. Finally the influence different
variables have on the predictive capabilities of the model are analyzes.<br/>
The whole report is generated by fitting a sum product network to the
data and extracting all information from this model.

In [5]:
printmd('''
---
## General statistical evaluation''')


---
## General statistical evaluation

In [6]:
reload(descr)
descr.data_description(spn, df)
printmd('''Below, the means and standard deviations of each feature are shown. Categorical 
features do not have a mean and a standard deviation, since they contain no ordering. Instead, 
the network returns NaN.''')

AttributeError: 'Sum' object has no attribute 'numFeatures'

In [ ]:
descr.means_table(spn)

In [ ]:
printmd('''In the following section, the marginal distributions for each feature is shown. This 
is the distribution of each feature without knowing anything about the other values.''')

In [ ]:
reload(descr)
descr.features_shown = 2

descr.show_feature_marginals(spn, dictionary)

In [ ]:
printmd('''
### Correlations

To get a sense of how the features relate to one another, the correlation between 
them is analyzed in the next section. The correlation denotes how strongly two features are 
linked. A high correlation (close to 1 or -1) means that two features are very closely related, 
while a correlation close to 0 means that there is no linear interdependency between the features.

The correlation is reported in a colored matrix, where blue denotes a negative and red denotes 
a positive correlation.''')

In [ ]:
descr.correlation_threshold = 0.4

corr = descr.correlation_description(spn)

In [ ]:
printmd('''The conditional distributions are the probabilities of the features, given 
a certain instance of a class. The joint probability functions of correlated variables 
are shown below to allow a more in-depth look into the dependency.''')

In [ ]:
reload(descr)

descr.correlation_threshold = 0
descr.feature_combinations = 2
descr.show_conditional = True

descr.categorical_correlations(spn, dictionary)

In [ ]:
printmd('''
---

## Cluster evaluation

To give an impression of the data representation as a whole, the complete network graph is 
shown below. The model is a tree, with a sum node at its center. The root of the tree is shown 
in white, while the sum and product nodes are green and blue respectively. Finally, all 
leaves are represented by red nodes.
''')

In [ ]:
p.plot_graph(spn=spn, fname='graph.png')
display(Image(filename='graph.png', width=400))

In [ ]:
printmd('''
The data model provides a clustering of the data points into groups in which features are 
independent. The groups extracted from the data are outlined below together with a short 
description of the data they cover. Each branch in the model represents one cluster found 
in the data model.

### Description of all clusters''')

In [ ]:
# possible values: 'all', 'big', int (leading to a random sample), list of nodes to be displayed
nodes = f.get_sorted_nodes(spn)

reload(descr)
descr.nodes = 'all'
descr.show_node_graphs = True

descr.node_introduction(spn, nodes)

In [ ]:
printmd('''
As stated above, each cluster captures a subgroup of the data. To show what variables are 
captured by which cluster, the means and variances for each feature and subgroup are plotted below. 
This highlights where the node has its focus.''')

In [ ]:
descr.features_shown = 2
descr.mean_threshold = 1
descr.variance_threshold = 2
descr.separation_threshold = 0.3

separations = descr.show_node_separation(spn, nodes)

In [ ]:
printmd('''
An analysis of the 
distribution of categorical variables is given below. If a cluster or a group of clusters 
capture a large fraction of the total likelihood of a categorical instance, they can be 
interpreted to represent this instance and the associated distribution.
''')

In [ ]:
descr.categoricals = 'all'

descr.node_categorical_description(spn, dictionary)

In [ ]:
printmd('''
### Correlations by cluster

Finally, since each node captures different interaction between the features, it is 
interesting to look at the correlations again, this time for the seperate nodes. Shallow 
nodes are omitted, because the correlation of independent variables is always 0.
''')

In [ ]:
descr.correlation_threshold = 0.3
descr.nodes = 1

descr.node_correlation(spn)

In [ ]:
printmd('''
---
## Predictive data analysis
''')

In [ ]:
reload(f)
numerical_data, categorical_data = f.get_categorical_data(spn, df, dictionary)

In [ ]:
printmd('''
After the cluster description, the data model is used to predict data points. To evaluate 
the performance of the model, the misclassification rate is shown below.

The classified data points are used to analyze more advanced patterns within the data, by looking
first at the misclassified points, and then at the classification results in total.
''')

In [ ]:
descr.classify = 'all'

misclassified, data_dict = descr.classification(spn, numerical_data, categorical_data)

In [ ]:
printmd('''
Below, the misclassified examples are explained using the clusters they are most assoiciated with.
For each instance, those clusters which form 90 % of the prediction are reported together eith the
representatives of these clusters.
''')

In [ ]:
# IMPORTANT: Only set use_shapley to true if you have a really powerful machine
reload(descr)
reload(p)

descr.use_shapley = False
descr.shapley_sample_size = 1
descr.misclassified_explanations = 1

descr.describe_misclassified(spn, dictionary, misclassified, data_dict, numerical_data)

In [ ]:
printmd('''
### Information gain through features

The following graphs highlight the relative importance of different features for a 
classification. It can show how different classes are predicted. For continuous and
discrete features, a high positvie or negative importance shows that changing this features
value positive or negative increases the predictions certainty.

For categorical values, positive and negative values highlight whether changing or keeping
this categorical value increases or decreasies the predictive certainty.
''')

In [ ]:
reload(descr)
reload(p)

descr.explanation_vector_threshold = 0
descr.explanation_vector_classes = None
descr.explanation_vectors_show = 'all'

expl_vectors = descr.explanation_vector_description(spn, dictionary, data_dict, categoricals)

In [ ]:
printmd(
'''
---

## Conclusion
'''
)

In [ ]:
reload(descr)
descr.print_conclusion(spn, dictionary, corr, nodes, separations, expl_vectors)

## Dive into the data

Use the Facets Interface to visualize data on your own. You can either load the dataset itself, or show the data as predicted by the model.

In [ ]:
# Load UCI census and convert to json for sending to the visualization
import pandas as pd
df = pd.read_csv(dataset)
jsonstr = df.to_json(orient='records')

# Display the Dive visualization for this data
from IPython.core.display import display, HTML

HTML_TEMPLATE = """<link rel="import" href="/nbextensions/facets-dist/facets-jupyter.html">
        <facets-dive id="elem" height="600"></facets-dive>
        <script>
          var data = {jsonstr};
          document.querySelector("#elem").data = data;
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=jsonstr)
display(HTML(html))

## Build your own queries

This notebook enables you to add your own analysis to the above. Maybe you are interested in drilling down into specific subclusters of the data, or you want to predict additional datapoint not represented in the training data.

In [ ]:
# get samples to predict
data_point = numerical_data[1:2]
# get the probability from the models joint probability function
proba = query(spn, data_point)


printmd(data_point)
printmd(query(spn, data_point))

You can also predict the probability of several data points at the same time.

In [ ]:
data_point = numerical_data[0:3]
proba = query(spn, data_point)

printmd(data_point)
printmd(proba)

If you are interested in predicting the probability of a class variable, you can use the following functions. The first one returns the probabilities of all possible class instances, the second one returns the instance with the highest probability.

In [ ]:
# get probabilities of all classes
proba = predict_proba(spn, categoricals[0], data_point)
# get the most likely class
predictions = predict(spn, categoricals[0], data_point)

for prob in proba:
    printmd(prob)
printmd(predictions)

The following code provides a more complicated example, showcasing how the data model can be used more in depth for a logistic regression task.

In [ ]:
# find the first continuous feature for a regression task
non_categorical = [i for i, name in enumerate(spn.featureNames) if name != 'categorical'][0]

# get sample data points
instances = numerical_data[0:5].copy()

# generate evidence for the regression. All features are used as predictors
evidence = instances.copy()
evidence[:,non_categorical] = np.nan

# get the prediction using the most probable explanation from the model
prediction = spn.root.mpe_eval(evidence)

#print the output
printmd('Predicted values: {}'.format(prediction[1][:,non_categorical]))
printmd('Real instances: {}'.format(instances[:,non_categorical]))